<a href="https://colab.research.google.com/github/146790g/Mode-Of-Action-Data/blob/master/MOA_MLP_2_Model_Fitting_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mode of Action

##Mechanisms of Action (MoA) Prediction
##Can you improve the algorithm that classifies drugs based on their biological activity?

#（その２： Model Fitting - No.1 ）

https://machinelearningmastery.com/multi-label-classification-with-deep-learning/



In [ ]:
%tensorflow_version 2.x

In [ ]:
%pwd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls -lh '/content/gdrive/My Drive/Colab Notebooks/MOA/data'

total 539M
-rw------- 1 root root 3.2M Sep  2 06:04 sample_submission.csv
-rw------- 1 root root  25M Sep  2 06:04 test_features.csv
-rw------- 1 root root 150M Sep  2 06:04 train_features.csv
-rw------- 1 root root  19M Sep  2 06:04 train_targets_nonscored.csv
-rw------- 1 root root 9.7M Sep  2 06:04 train_targets_scored.csv
-rw------- 1 root root  32M Nov  3 08:01 X_fp_test
-rw------- 1 root root 190M Nov  3 08:01 X_fp_train
-rw------- 1 root root  38M Nov  3 08:01 y
-rw------- 1 root root  74M Nov  3 08:01 y0


# Module Load

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Adding iterative-stratification 
# Select add data from the right menu and search for iterative-stratification, then add it to your kernel.
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


from time import time
import datetime
import gc
import pickle

import numpy as np
import pandas as pd 

# ML tools 
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import tensorflow as tf 
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from sklearn.metrics import log_loss
from tensorflow_addons.layers import WeightNormalization
# Setting random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Visualization tools
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('white')
sns.set(font_scale=1.2)




# Data Load

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/content/gdrive/My Drive/Colab Notebooks/MOA/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



data_dir = '/content/gdrive/My Drive/Colab Notebooks/MOA/data'

/content/gdrive/My Drive/Colab Notebooks/MOA/data/test_features.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/sample_submission.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_features.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_targets_scored.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/train_targets_nonscored.csv
/content/gdrive/My Drive/Colab Notebooks/MOA/data/X_fp_train
/content/gdrive/My Drive/Colab Notebooks/MOA/data/X_fp_test
/content/gdrive/My Drive/Colab Notebooks/MOA/data/y
/content/gdrive/My Drive/Colab Notebooks/MOA/data/y0


In [ ]:
with open(os.path.join(data_dir,'X_fp_train'),'rb') as web:
  X_fp_train=pickle.load(web)

   
with open(os.path.join(data_dir,'X_fp_test'),'rb') as web:
  X_fp_test=pickle.load(web)
    
with open(os.path.join(data_dir,'y'),'rb') as web:
  y=pickle.load(web)
    
with open(os.path.join(data_dir,'y0'),'rb') as web:
  y0=pickle.load(web)
    
X_fp_train=X_fp_train.drop('sig_id',axis=1)
X_fp_test=X_fp_test.drop('sig_id',axis=1)
y=y.drop('sig_id',axis=1)
y0=y0.drop('sig_id',axis=1)

display(X_fp_train.head(3))
print(X_fp_train.shape)
print(type(X_fp_train))

display(X_fp_test.head(3))
print(X_fp_test.shape)

display(y.head(3))
print(y.shape)

display(y0.head(3))
print(y0.shape)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,feature_1024,feature_1025,feature_1026,feature_1027,feature_1028,feature_1029,feature_1030,feature_1031,feature_1032,feature_1033,feature_1034,feature_1035,feature_1036,feature_1037,feature_1038,cp_type,cp_time,cp_dose
0,-0.755294,0.214796,-0.774511,0.705349,1.564580,-0.194968,-0.276944,0.363393,-0.444533,1.805324,-0.371021,-0.479669,1.294179,1.375143,0.051658,1.172079,-0.440924,-0.062191,-0.515950,-0.897857,-0.789403,0.331908,0.013708,0.731217,0.280174,0.726955,1.007077,0.758206,-0.463151,-0.205948,-0.729595,-0.068265,-0.926625,-0.757403,-1.410264,-0.018377,0.603804,0.831659,-0.407849,-1.489034,...,0.883171,1.403209,0.224713,0.076150,0.560326,1.309312,0.946857,-0.801600,1.908884,0.940878,-0.542073,0.191524,2.194069,0.248358,0.780708,-0.145965,-1.125801,1.049904,0.410974,0.364819,1.291804,0.835350,-0.240101,1.021706,-0.499652,0.317989,0.545662,0.641339,4.917651,1.538645,-1.560099,1.127068,0.908941,-1.080358,-0.253258,0.115213,-0.226061,0,1,0
1,-0.186773,0.374643,1.176402,-0.652299,-0.546638,-0.403447,-2.325903,0.645634,-0.485088,-0.048936,1.210023,0.419276,-1.275817,-0.425276,0.132044,-0.890770,-1.146702,-1.251700,-0.274084,-1.058294,-0.821554,1.029957,0.843801,0.088734,0.644651,-0.194701,-0.240824,1.799069,-1.067548,-0.836341,-0.634064,-0.140889,-0.423926,0.252623,-0.830175,1.399657,-1.112732,-1.097712,0.456949,0.012281,...,0.671709,0.853389,0.170340,1.178457,0.759411,-0.406442,-0.113337,0.625945,-0.355584,1.172363,0.868525,0.500542,0.214436,0.645295,1.394635,1.211742,-0.528146,0.507482,-0.520372,1.127405,0.716111,0.054620,0.412012,0.744215,0.210242,0.179684,0.919161,1.165833,5.090695,-0.368828,0.013911,0.994664,-0.502353,0.673057,-0.272862,0.455034,-1.085957,0,3,0
2,0.351229,-0.155308,-0.613250,0.255053,-1.762152,0.333321,-0.434423,-0.484560,0.857189,0.514641,-0.132592,-0.834668,1.793695,-1.108673,1.197317,-0.086980,0.054445,1.397045,-1.129795,-1.353153,1.041231,-0.465987,-0.838953,0.284006,-0.377608,0.284396,0.769001,-0.964506,0.009738,0.077140,0.418757,-0.766888,-0.551620,-0.150097,-1.309084,0.404371,0.432542,0.741155,-1.588311,-1.132478,...,0.750383,0.194436,0.352697,1.526166,-0.084895,-0.141983,0.152105,0.089034,-1.015340,-0.852661,-0.151391,-0.742431,0.436651,0.160259,0.137482,-0.415975,0.456803,-0.078962,-0.828896,-0.740965,0.953239,0.053633,-1.213056,-0.394118,-0.758652,-0.277635,-1.123088,1.089235,-1.352416,0.294769,-0.291118,-0.218369,-0.098231,-0.711736,0.893185,0.039182,-1.085265,0,2,0


(23814, 1042)
<class 'pandas.core.frame.DataFrame'>


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,feature_1024,feature_1025,feature_1026,feature_1027,feature_1028,feature_1029,feature_1030,feature_1031,feature_1032,feature_1033,feature_1034,feature_1035,feature_1036,feature_1037,feature_1038,cp_type,cp_time,cp_dose
0,-0.755294,0.214796,-0.774511,0.705349,1.564580,-0.194968,-0.276944,0.363393,-0.444533,1.805324,-0.371021,-0.479669,1.294179,1.375143,0.051658,1.172079,-0.440924,-0.062191,-0.515950,-0.897857,-0.789403,0.331908,0.013708,0.731217,0.280174,0.726955,1.007077,0.758206,-0.463151,-0.205948,-0.729595,-0.068265,-0.926625,-0.757403,-1.410264,-0.018377,0.603804,0.831659,-0.407849,-1.489034,...,-0.033252,-0.492298,-1.032975,-0.390962,0.571419,-0.646490,0.413347,0.008858,-0.137114,0.447321,-0.129989,-1.030313,-1.073490,0.026809,0.024562,-0.021795,-0.667219,-0.398334,0.116890,1.194732,-0.195260,-0.298039,-0.301677,-0.128405,-0.036089,-0.094841,0.153376,-0.920303,-0.804259,-1.899650,0.657893,-0.338125,0.099395,-0.800232,0.523428,-0.393640,-0.745169,0,1,0
1,-0.186773,0.374643,1.176402,-0.652299,-0.546638,-0.403447,-2.325903,0.645634,-0.485088,-0.048936,1.210023,0.419276,-1.275817,-0.425276,0.132044,-0.890770,-1.146702,-1.251700,-0.274084,-1.058294,-0.821554,1.029957,0.843801,0.088734,0.644651,-0.194701,-0.240824,1.799069,-1.067548,-0.836341,-0.634064,-0.140889,-0.423926,0.252623,-0.830175,1.399657,-1.112732,-1.097712,0.456949,0.012281,...,0.065126,-1.219456,-0.616402,-0.162918,-0.788433,-0.749537,-0.921389,0.075357,-1.437453,-1.254469,-0.861142,-0.740949,-1.316315,-0.915945,0.745559,0.840110,-0.565178,0.207971,-0.183415,-0.269845,-1.059129,-1.317859,-0.466468,-0.653500,-0.015480,-0.621168,-1.191952,-0.880456,-5.397308,-0.456508,0.290936,-1.095456,-0.612847,1.042001,0.832567,1.149697,0.106494,0,3,0
2,0.351229,-0.155308,-0.613250,0.255053,-1.762152,0.333321,-0.434423,-0.484560,0.857189,0.514641,-0.132592,-0.834668,1.793695,-1.108673,1.197317,-0.086980,0.054445,1.397045,-1.129795,-1.353153,1.041231,-0.465987,-0.838953,0.284006,-0.377608,0.284396,0.769001,-0.964506,0.009738,0.077140,0.418757,-0.766888,-0.551620,-0.150097,-1.309084,0.404371,0.432542,0.741155,-1.588311,-1.132478,...,0.723740,0.971419,1.352093,0.638580,1.507330,-0.666720,-0.443788,0.696583,1.710023,1.152024,0.252428,-0.605626,2.125087,0.281960,1.824982,-0.759893,0.730202,-0.204462,-0.328362,0.463214,-1.233935,1.310102,-1.429491,-1.093671,1.239167,1.577578,0.681627,-0.241098,3.589625,1.289070,0.156473,-0.164602,-1.030081,2.044905,-0.786719,-0.974288,1.196178,1,1,0


(3982, 1042)


,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(23814, 206)


,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,aldehyde_dehydrogenase_activator,aldose_reductase_inhibitor,alpha_mannosidase_inhibitor,ampk_inhibitor,androgen_biosynthesis_inhibitor,angiotensin_receptor_agonist,antacid,anthelmintic,antihypertensive,antimetabolite,antipruritic,antirheumatic_drug,antiseptic,antispasmodic,antithyroid_agent,antitussive,anxiolytic,ap_inhibitor,apolipoprotein_expression_enhancer,apoptosis_inhibitor,arf_inhibitor,aryl_hydrocarbon_receptor_agonist,aryl_hydrocarbon_receptor_antagonist,aspartic_protease_inhibitor,atherogenesis_inhibitor,atherosclerosis_formation_inhibitor,atp-sensitive_potassium_channel_agonist,atp-sensitive_potassium_channel_inhibitor,atp_channel_activator,atp_channel_blocker,atp_citrase_lyase_inhibitor,...,thromboxane_synthase_inhibitor,thyroid_hormone_inhibitor,thyroid_hormone_stimulant,thyrotropin_releasing_hormone_receptor_agonist,tie_inhibitor,tissue_transglutaminase_inhibitor,topical_anesthetic,topical_sunscreen_agent,trace_amine_associated_receptor_agonist,trace_amine_associated_receptor_antagonist,trail_modulator,transient_receptor_potential_channel_agonist,triacylglycerol_lipase_inhibitor,tricyclic_antidepressant,tryptophan_hydroxylase_inhibitor,tyrosinase_inhibitor,tyrosine_hydroxylase_inhibitor,tyrosine_phosphatase_inhibitor,ubiquitin-conjugating_enzyme_inhibitor,ubiquitin_ligase_inhibitor,urease_inhibitor,uric_acid_diuretic,uricase_inhibitor,uricosuric,urotensin_receptor_agonist,urotensin_receptor_antagonist,vasoconstrictor,vasodilator,vasopressin_receptor_agonist,vasopressin_receptor_antagonist,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(23814, 402)


# Define Model

In [ ]:
X_fp_train.shape[1],y.shape[1]


(1042, 206)

In [ ]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda,Activation,Dropout,Flatten,Dense
import keras.backend as K
from tensorflow import keras

def create_model(n_inputs,n_outputs):
  model = keras.Sequential([
                            Input(shape=(n_inputs,)),
                            Dense(800, activation='relu'),
                            Dense(400, activation='relu'),
                            Dense(n_outputs, activation='sigmoid')])
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

#model=create_model(X_fp_train.shape[1],y.shape[1])

#model.summary()

In [ ]:
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = create_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# make a prediction on the test set
		yhat = model.predict(X_test)
		# round probabilities to class labels
		yhat = yhat.round()
		# calculate accuracy
		acc = accuracy_score(y_test, yhat)
		# store result
		print('>%.3f' % acc)
		results.append(acc)
	return results


In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

# evaluate model
X=X_fp_train.astype('float64').values
y=y.astype('float64').values
results = evaluate_model(X, y)


# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

>0.398
>0.395
>0.407
>0.389
>0.403
>0.385
>0.384
>0.387
>0.391
>0.394
>0.380
>0.396
>0.413
>0.395
>0.388
>0.404
>0.395
>0.382
>0.407
>0.375
>0.393
>0.397
>0.396
>0.397
>0.396
>0.388
>0.375
>0.401
>0.380
>0.410
Accuracy: 0.393 (0.010)


# Q.E.D. 